# Desafío RabbitMQ: Sistema de Logística Global
### Patrón Topic Exchange

---

## 1. El Escenario

Imagina que trabajas para una empresa de e-commerce internacional llamada **"FastShip"**. La empresa envía miles de paquetes cada minuto. Para que el sistema sea eficiente, los mensajes de los nuevos pedidos deben llegar solo a los departamentos interesados.

Cada mensaje de pedido tiene una **Routing Key** con el siguiente formato:
  
`<pais>.<metodo_envio>.<prioridad>`

*   **País:** `es` (España), `mx` (México), `us` (EE.UU.), `uk` (Reino Unido).
*   **Método de Envío:** `standard`, `express`, `overnight`.
*   **Prioridad:** `baja`, `media`, `alta`.

**Ejemplo de Routing Key:** `mx.express.alta` (Un pedido urgente en México).

## 2. Concepto: Topic Exchange

A diferencia del intercambio directo (Direct Exchange), el **Topic Exchange** permite realizar filtrados complejos usando comodines:

*   `*` (asterisco) sustituye exactamente **una palabra**.
*   `#` (almohadilla) sustituye **cero o más palabras**.

### Ejemplos de filtrado (Binding Keys):
*   `es.#`: Recibe todo lo que ocurra en España (`es.standard.baja`, `es.express.media`, etc.).
*   `#.alta`: Recibe todos los pedidos de prioridad alta de cualquier país y cualquier método.
*   `*.express.*`: Recibe todos los pedidos Express sin importar el país o la prioridad.

## 3. Código Base: El Productor de Logística

Este código simula la creación de pedidos aleatorios. **Ejecútalo para empezar a enviar datos a la cola.**

In [ ]:
import pika
import random
import time

def publicar_pedidos():
    connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
    channel = connection.channel()

    # Declaramos un exchange de tipo 'topic'
    channel.exchange_declare(exchange='logistica_global', exchange_type='topic')

    paises = ['es', 'mx', 'us', 'uk']
    metodos = ['standard', 'express', 'overnight']
    prioridades = ['baja', 'media', 'alta']

    print(" [x] Enviando 20 pedidos aleatorios...")

    for i in range(20):
        routing_key = f"{random.choice(paises)}.{random.choice(metodos)}.{random.choice(prioridades)}"
        mensaje = f"Pedido #{i+1} - Detalles: {routing_key}"
        
        channel.basic_publish(
            exchange='logistica_global',
            routing_key=routing_key,
            body=mensaje
        )
        print(f" [v] Enviado: '{routing_key}'")
        time.sleep(0.5)

    connection.close()

if __name__ == "__main__":
    publicar_pedidos()

--- 
## 4. EL DESAFÍO

Tu misión es crear **tres consumidores especializados** (puedes crear celdas nuevas o archivos `.py`). Cada uno debe tener una lógica de filtrado distinta.

### Tarea 1: El Gerente Regional (México)
Crea un consumidor que reciba **absolutamente todos** los pedidos realizados en México (`mx`).
*   **Binding Key sugerida:** `mx.#` 

### Tarea 2: El Monitor de Urgencias (Global)
Crea un consumidor que solo se interese por los pedidos de **Prioridad Alta**, sin importar de qué país vengan o cómo se envíen.
*   **Binding Key sugerida:** `#.alta` 

### Tarea 3: Logística Express Europea
Crea un consumidor que reciba pedidos que sean tanto de **España** (`es`) como de tipo **Express** o **Overnight**.
*   *Pista:* Tal vez necesites hacer dos "bindings" diferentes para la misma cola en este consumidor.

---

## 5. Ejemplo de Código para el Consumidor

Usa esta estructura como guía para tus tres tareas. Solo debes cambiar la `routing_key` en el `queue_bind`.

In [ ]:
import pika

def iniciar_consumidor(binding_key):
    connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
    channel = connection.channel()

    channel.exchange_declare(exchange='logistica_global', exchange_type='topic')

    # Creamos una cola exclusiva para este consumidor
    result = channel.queue_declare(queue='', exclusive=True)
    nombre_cola = result.method.queue

    # AQUÍ ES DONDE OCURRE LA MAGIA DEL FILTRADO
    channel.queue_bind(
        exchange='logistica_global', 
        queue=nombre_cola, 
        routing_key=binding_key
    )

    print(f" [*] Esperando mensajes con filtro: {binding_key}. CTRL+C para salir")

    def callback(ch, method, properties, body):
        print(f" [x] RECIBIDO ({method.routing_key}): {body.decode()}")

    channel.basic_consume(queue=nombre_cola, on_message_callback=callback, auto_ack=True)
    channel.start_consuming()

# Cambia esto para probar cada Tarea:
# iniciar_consumidor('mx.#')

## 6. Resultado Esperado

Si lo lograste correctamente:

1.  Al enviar un mensaje `mx.standard.baja`, **SOLO** debería recibirlo el Consumidor 1 (México).
2.  Al enviar un mensaje `us.standard.alta`, **SOLO** debería recibirlo el Consumidor 2 (Urgencias).
3.  Al enviar un mensaje `es.express.alta`, deberían recibirlo **TANTO** el Consumidor 2 (Urgencias) como el Consumidor 3 (Logística Express Europea).
4.  Si un mensaje es `uk.standard.baja`, **NINGÚN** consumidor debería recibirlo (se descarta).

Productor:

In [ ]:
import pika
import random
import time

def publicar_pedidos():
    connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
    channel = connection.channel()

    # Declaramos un exchange de tipo 'topic'
    channel.exchange_declare(exchange='logistica_global', exchange_type='topic')

    paises = ['es', 'mx', 'us', 'uk']
    metodos = ['standard', 'express', 'overnight']
    prioridades = ['baja', 'media', 'alta']

    print(" [x] Enviando 20 pedidos aleatorios...")

    for i in range(20):
        routing_key = f"{random.choice(paises)}.{random.choice(metodos)}.{random.choice(prioridades)}"
        mensaje = f"Pedido #{i+1} - Detalles: {routing_key}"
        
        channel.basic_publish(
            exchange='logistica_global',
            routing_key=routing_key,
            body=mensaje
        )
        print(f" [v] Enviado: '{routing_key}'")
        time.sleep(0.5)

    connection.close()

if __name__ == "__main__":
    publicar_pedidos()

Consumidor México

In [ ]:
import pika

def consumidor_mexico():
    binding_key = "mx.#"

    connection = pika.BlockingConnection(pika.ConnectionParameters("localhost"))
    channel = connection.channel()

    channel.exchange_declare(exchange="logistica_global", exchange_type="topic")

    result = channel.queue_declare(queue="", exclusive=True)
    cola = result.method.queue

    channel.queue_bind(exchange="logistica_global", queue=cola, routing_key=binding_key)

    print(f"🇲🇽 [*] Gerente Regional (MX) esperando: {binding_key}  CTRL+C para salir")

    def callback(ch, method, properties, body):
        print(f"🇲🇽 [x] RECIBIDO ({method.routing_key}): {body.decode()}")

    channel.basic_consume(queue=cola, on_message_callback=callback, auto_ack=True)
    channel.start_consuming()

# Ejecuta:
consumidor_mexico()


Consumidor global

In [ ]:
import pika

def consumidor_urgencias_global():
    binding_key = "#.alta"

    connection = pika.BlockingConnection(pika.ConnectionParameters("localhost"))
    channel = connection.channel()

    channel.exchange_declare(exchange="logistica_global", exchange_type="topic")

    result = channel.queue_declare(queue="", exclusive=True)
    cola = result.method.queue

    channel.queue_bind(exchange="logistica_global", queue=cola, routing_key=binding_key)

    print(f"🚨 [*] Monitor de Urgencias (GLOBAL) esperando: {binding_key}  CTRL+C para salir")

    def callback(ch, method, properties, body):
        print(f"🚨 [x] RECIBIDO ({method.routing_key}): {body.decode()}")

    channel.basic_consume(queue=cola, on_message_callback=callback, auto_ack=True)
    channel.start_consuming()

# Ejecuta:
consumidor_urgencias_global()


Consumidor España

In [ ]:
import pika

def consumidor_express_europa_es():
    binding_keys = ["es.express.*", "es.overnight.*"]

    connection = pika.BlockingConnection(pika.ConnectionParameters("localhost"))
    channel = connection.channel()

    channel.exchange_declare(exchange="logistica_global", exchange_type="topic")

    result = channel.queue_declare(queue="", exclusive=True)
    cola = result.method.queue

    # 2 bindings a la MISMA cola (esto cumple la pista)
    for bk in binding_keys:
        channel.queue_bind(exchange="logistica_global", queue=cola, routing_key=bk)

    print(f"🇪🇸⚡ [*] Logística Express Europea (ES) esperando: {binding_keys}  CTRL+C para salir")

    def callback(ch, method, properties, body):
        print(f"🇪🇸⚡ [x] RECIBIDO ({method.routing_key}): {body.decode()}")

    channel.basic_consume(queue=cola, on_message_callback=callback, auto_ack=True)
    channel.start_consuming()

# Ejecuta:
consumidor_express_europa_es()
